In [2]:
#Using the Histogram to perform Classification!

In [3]:
import cv2 as cv
import os
import pandas as pd
import numpy as np
import json
import pickle
import time
import winsound

In [4]:
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.utils import shuffle


In [5]:
start_program = time.time()
duration = 800  # milliseconds
freq = 300 # Hz
winsound.Beep(freq, duration)

In [6]:
n_cluster = 1500
car_name = "_".join(("Car_Hist_MinMax", "C"+str(n_cluster)+".csv"))
noise_name = "_".join(("Noise_Hist_MinMax","C"+str(n_cluster)+".csv"))
car_name

'Car_Hist_MinMax_C1500.csv'

In [7]:
car_df = pd.read_csv(car_name, sep = ',', index_col = False)
noise_df = pd.read_csv(noise_name, sep = ',', index_col = False)

In [8]:
#Scaling the dataset - Already Scaled!
# car_df = car_df/500
# noise_df = noise_df/500

In [9]:
car_df['y'] = 1
noise_df['y'] = 0

In [10]:
df = pd.concat([car_df, noise_df])
df.head()

0     1         2         3    4    5    6     7    8         9  ...  \
0  0.0  0.00  0.000000  0.166667  0.0  0.0  0.0  0.00  0.0  0.000000  ...   
1  0.0  0.50  0.000000  0.000000  0.0  0.0  0.0  0.00  0.0  0.000000  ...   
2  0.0  0.25  0.000000  0.000000  0.2  0.0  0.0  0.25  0.2  0.142857  ...   
3  0.0  0.25  0.000000  0.000000  0.4  0.0  0.0  0.00  0.0  0.142857  ...   
4  0.0  0.00  0.166667  0.000000  0.0  0.0  0.0  0.00  0.6  0.285714  ...   

   1491      1492  1493  1494      1495  1496      1497  1498      1499  y  
0   0.4  0.142857   0.0   0.0  0.000000   0.0  0.000000   0.0  0.000000  1  
1   0.0  0.000000   0.0   0.2  0.000000   0.2  0.000000   0.0  0.166667  1  
2   0.0  0.000000   0.0   0.2  0.333333   0.2  0.000000   0.0  0.000000  1  
3   0.0  0.000000   0.0   0.0  0.000000   0.2  0.714286   0.0  0.000000  1  
4   0.0  0.000000   0.2   0.0  0.166667   0.2  0.142857   0.0  0.000000  1  

[5 rows x 1501 columns]

In [11]:
# Didn't work!
# l = len(df)
# index = np.random.permutation(l)
#df = df.iloc[np.random.permutation(len(df))] 
df.head()

0     1         2         3    4    5    6     7    8         9  ...  \
0  0.0  0.00  0.000000  0.166667  0.0  0.0  0.0  0.00  0.0  0.000000  ...   
1  0.0  0.50  0.000000  0.000000  0.0  0.0  0.0  0.00  0.0  0.000000  ...   
2  0.0  0.25  0.000000  0.000000  0.2  0.0  0.0  0.25  0.2  0.142857  ...   
3  0.0  0.25  0.000000  0.000000  0.4  0.0  0.0  0.00  0.0  0.142857  ...   
4  0.0  0.00  0.166667  0.000000  0.0  0.0  0.0  0.00  0.6  0.285714  ...   

   1491      1492  1493  1494      1495  1496      1497  1498      1499  y  
0   0.4  0.142857   0.0   0.0  0.000000   0.0  0.000000   0.0  0.000000  1  
1   0.0  0.000000   0.0   0.2  0.000000   0.2  0.000000   0.0  0.166667  1  
2   0.0  0.000000   0.0   0.2  0.333333   0.2  0.000000   0.0  0.000000  1  
3   0.0  0.000000   0.0   0.0  0.000000   0.2  0.714286   0.0  0.000000  1  
4   0.0  0.000000   0.2   0.0  0.166667   0.2  0.142857   0.0  0.000000  1  

[5 rows x 1501 columns]

In [12]:
#df = shuffle(df)
X = df.drop(columns = ['y'])
y = df['y']

#Do below when you have Columns 500 and 501
# X = df.drop(columns = ['y', '500','501'])
# y = df['y']
display(X.head())
display(y.head())

0     1         2         3    4    5    6     7    8         9  ...  \
0  0.0  0.00  0.000000  0.166667  0.0  0.0  0.0  0.00  0.0  0.000000  ...   
1  0.0  0.50  0.000000  0.000000  0.0  0.0  0.0  0.00  0.0  0.000000  ...   
2  0.0  0.25  0.000000  0.000000  0.2  0.0  0.0  0.25  0.2  0.142857  ...   
3  0.0  0.25  0.000000  0.000000  0.4  0.0  0.0  0.00  0.0  0.142857  ...   
4  0.0  0.00  0.166667  0.000000  0.0  0.0  0.0  0.00  0.6  0.285714  ...   

   1490  1491      1492  1493  1494      1495  1496      1497  1498      1499  
0   0.0   0.4  0.142857   0.0   0.0  0.000000   0.0  0.000000   0.0  0.000000  
1   0.0   0.0  0.000000   0.0   0.2  0.000000   0.2  0.000000   0.0  0.166667  
2   0.0   0.0  0.000000   0.0   0.2  0.333333   0.2  0.000000   0.0  0.000000  
3   0.0   0.0  0.000000   0.0   0.0  0.000000   0.2  0.714286   0.0  0.000000  
4   0.0   0.0  0.000000   0.2   0.0  0.166667   0.2  0.142857   0.0  0.000000  

[5 rows x 1500 columns]

0    1
1    1
2    1
3    1
4    1
Name: y, dtype: int64

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X.values, y.values, test_size = 0.05, shuffle = True)

In [14]:
Y_test[:5]

array([1, 0, 0, 0, 1], dtype=int64)

In [15]:
len(X_train)

12655

In [16]:
model_SVC = SVC()

In [17]:
start_time = time.time()
model_SVC.fit(X = X_train, y = Y_train)
print(time.time() - start_time)

443.38231086730957


In [18]:
Y_predict = model_SVC.predict(X_test)

In [19]:
f1_score(y_true = Y_test, y_pred = Y_predict)
#Previous Iteration: 2.1% - 20%
#After all corrections! v1: 99% -> v3: 0.997

0.8325652841781875

In [20]:
accuracy_score(y_true = Y_test, y_pred = Y_predict)
#Previous Iteration: 2.1% - 20%
#After all corrections! v1: 0.994 -> v3: 0.997
# For C1000: SVC - train accuracy: 0.998
# For C800: SVC - train accuracy: 0.998
# For C500: SVC - train accuracy: 0.998
# For C1000: SVC - train accuracy: 0.995

0.8365817091454273

In [21]:
#Dumping the trained model!
SVC_name = "_".join(("SVC","VoW_MinMax","C"+str(n_cluster)+".sav"))
print(SVC_name)
pickle.dump(model_SVC, open(SVC_name, 'wb'))

SVC_VoW_MinMax_C1500.sav


In [22]:
print("Program Time: {}".format(time.time() - start_program))
winsound.Beep(freq, duration)
#C1000: 55 sec
#C800:  30 sec
#C500: 30 sec

Program Time: 467.3774528503418


In [ ]:
Stop!

In [17]:
#Testing the prediction accuracy with Validation Data!
#Let's import all the feature descriptors!
path_dir = os.getcwd()
valid_dir = "\\".join((path_dir, "ORB_Dataset", "val"))
valid_files = os.listdir(valid_dir)
valid_car_dir = "\\".join((valid_dir, valid_files[0]))
valid_noise_dir = "\\".join((valid_dir, valid_files[1]))


In [18]:
valid_noise_dir

'i:\\WPI\\Fall 2020\\Computer Vision - RBE 549\\CV Project\\BoW_Visual\\ORB_Dataset\\val\\val_no_car_ORB.json'

In [19]:
start_time = time.time()
valid_car_descriptors = []
with open(valid_car_dir, 'r') as fil_name:
    valid_car_descriptors = json.load(fil_name)
print(time.time() - start_time)

start_time = time.time()
valid_noise_descriptors = []
with open(valid_noise_dir, 'r') as fil_name:
    valid_noise_descriptors = json.load(fil_name)
print(time.time() - start_time)

2.4884555339813232


In [22]:
def cluster_labels(pickle_file, model):
    start_time = time.time()
    #pickle_file = car_descriptors
    
    img_cluster = []
    for img_desc in pickle_file:
        cluster_desc = []
        if len(img_desc)> 0:
            #for desc in img_desc:
            cluster_desc = model.predict(img_desc)
            img_cluster.append(cluster_desc)
    print(time.time() - start_time)
    return(img_cluster)

In [21]:
#Load the model
#pickle.loads(kmeans_batch, open("KMeans_All_Trial_1.sav", 'wb'))

with open("KMeans_c_800_b_32_rs_5_v1.sav", 'rb') as f_name:
    kmeans_batch = pickle.load(f_name)

In [25]:
valid_car_clusters = np.array(cluster_labels(pickle_file = valid_car_descriptors, model = kmeans_batch))
valid_noise_clusters = np.array(cluster_labels(pickle_file = valid_noise_descriptors, model = kmeans_batch))

14.213318824768066
14.557066440582275


In [26]:
display(np.shape(valid_car_clusters))
display(np.shape(valid_noise_clusters))

(818,)

(847,)

In [27]:
def cluster_histogram(img_clusters, n_clusters = 500):
    start_time = time.time()
    hist_arr = []
    for img in img_clusters:
        hist = np.zeros(n_clusters)
        for cluster in img:
            hist[cluster] += 1
        hist_arr.append(hist)
    print(time.time() - start_time)
    return(hist_arr)

In [44]:
n_cluster = 800
car_hist = cluster_histogram(img_clusters = valid_car_clusters, n_clusters = n_cluster)
noise_hist = cluster_histogram(img_clusters = valid_noise_clusters, n_clusters = n_cluster)

0.3223090171813965
0.35868382453918457


In [45]:
valid_car_hist = pd.DataFrame(car_hist)
sscaler = StandardScaler()
valid_car_hist_ss = pd.DataFrame(sscaler.fit_transform(valid_car_hist))
print(np.max(car_df_v1[0]))
mm_scalar = MinMaxScaler()
valid_car_hist_mm = pd.DataFrame(mm_scalar.fit_transform(valid_car_hist))

3.0


In [47]:
valid_noise_hist = pd.DataFrame(noise_hist)
sscaler = StandardScaler()
valid_noise_hist_ss = pd.DataFrame(sscaler.fit_transform(valid_noise_hist))
mm_scalar = MinMaxScaler()
valid_noise_hist_mm = pd.DataFrame(mm_scalar.fit_transform(valid_noise_hist))

In [49]:
sample_car_mm = valid_car_hist_mm
sample_noise_mm = valid_noise_hist_mm

sample_car_ss = valid_car_hist_ss
sample_noise_ss = valid_noise_hist_ss

In [50]:
# It happens that sometimes there are no freq found for that particular cluster
print(np.any(sample_car_mm.isna())) 
print(np.any(sample_noise_mm.isna()))

False
False


In [52]:
sample_car_mm.to_csv("Valid_Car_Hist_MinMax_Max800_v1.csv", sep = ',', index= False)
sample_car_ss.to_csv("Valid_Car_Hist_SS_Max800_v1.csv", sep = ',', index= False)
sample_noise_mm.to_csv("Valid_Noise_Hist_MinMax_Max800_v1.csv", sep = ',', index= False)
sample_noise_ss.to_csv("Valid_Noise_Hist_SS_Max00_v1.csv", sep = ',', index= False)

In [53]:
len(sample_car_mm)

818

In [54]:
valid_car_df = sample_car_mm
valid_noise_df = sample_noise_mm
valid_car_df['y'] = 1
valid_noise_df['y'] = 0
valid_df = pd.concat([valid_car_df, valid_noise_df])
valid_df

0         1         2     3         4         5       6      7  \
0    0.000000  0.000000  0.125000  0.00  0.000000  0.300000  0.1000  0.000   
1    0.333333  0.000000  0.125000  0.00  0.000000  0.100000  0.0000  0.250   
2    0.000000  0.000000  0.125000  0.00  0.500000  0.000000  0.0000  0.000   
3    0.000000  0.222222  0.250000  0.00  0.000000  0.000000  0.1000  0.000   
4    0.666667  0.111111  0.000000  0.00  0.000000  0.100000  0.0000  0.000   
..        ...       ...       ...   ...       ...       ...     ...    ...   
842  0.000000  0.000000  0.071429  0.25  0.000000  0.045455  0.0625  0.125   
843  0.000000  0.000000  0.000000  0.25  0.000000  0.000000  0.1250  0.125   
844  0.000000  0.600000  0.000000  0.00  0.142857  0.000000  0.0000  0.000   
845  0.000000  0.000000  0.000000  0.25  0.000000  0.000000  0.0625  0.250   
846  0.000000  0.000000  0.000000  0.25  0.000000  0.181818  0.1875  0.125   

         8         9  ...   791       792       793       794       795  \
0    0.000  0.000000  ...  0.00  0.100000  0.166667  0.000000  0.000000   
1    0.000  0.000000  ...  0.00  0.000000  0.000000  0.142857  0.000000   
2    0.500  0.600000  ...  0.50  0.000000  0.000000  0.142857  0.080000   
3    0.000  0.200000  ...  0.00  0.000000  0.666667  0.142857  0.040000   
4    0.250  0.000000  ...  0.25  0.000000  0.000000  0.142857  0.000000   
..     ...       ...  ...   ...       ...       ...       ...       ...   
842  0.250  0.000000  ...  0.25  0.166667  0.000000  0.000000  0.000000   
843  0.250  0.000000  ...  0.00  0.166667  0.000000  0.000000  0.176471   
844  0.000  0.166667  ...  0.00  0.000000  0.166667  0.000000  0.000000   
845  0.000  0.000000  ...  0.00  0.166667  0.166667  0.000000  0.000000   
846  0.125  0.500000  ...  0.00  0.500000  0.000000  0.000000  0.000000   

          796       797  798  799  y  
0    0.000000  0.333333  0.0  0.2  1  
1    0.285714  0.166667  0.0  0.0  1  
2    0.000000  0.333333  0.0  0.4  1  
3    0.000000  0.000000  0.0  0.0  1  
4    0.000000  0.000000  0.0  0.2  1  
..        ...       ...  ...  ... ..  
842  0.222222  0.000000  0.0  0.0  0  
843  0.000000  0.250000  0.3  0.0  0  
844  0.111111  0.000000  0.2  0.0  0  
845  0.000000  0.250000  0.2  0.0  0  
846  0.000000  0.000000  0.1  0.0  0  

[1665 rows x 801 columns]

In [55]:
#df = shuffle(df)
X_valid = valid_df.drop(columns = ['y'])
y = valid_df['y']

#Do below when you have Columns 500 and 501
# X = df.drop(columns = ['y', '500','501'])
# y = df['y']
display(X.head())
display(y.head())

0      1         2    3      4         5         6         7    8     9  \
0  0.0  0.000  0.142857  0.0  0.125  0.083333  0.071429  0.090909  0.0  0.00   
1  0.2  0.000  0.357143  0.0  0.250  0.000000  0.071429  0.000000  0.0  0.00   
2  0.2  0.000  0.142857  0.0  0.125  0.000000  0.071429  0.000000  0.0  0.25   
3  0.2  0.000  0.142857  0.0  0.000  0.000000  0.000000  0.000000  0.0  0.25   
4  0.2  0.125  0.142857  0.0  0.250  0.166667  0.000000  0.000000  0.0  0.00   

   ...       790    791       792    793  794       795    796       797  \
0  ...  0.000000  0.125  0.000000  0.000  0.0  0.000000  0.000  0.000000   
1  ...  0.000000  0.000  0.000000  0.000  0.0  0.016949  0.125  0.000000   
2  ...  0.166667  0.000  0.285714  0.000  0.0  0.008475  0.125  0.111111   
3  ...  0.000000  0.000  0.142857  0.375  0.0  0.008475  0.000  0.555556   
4  ...  0.000000  0.250  0.142857  0.000  0.0  0.008475  0.000  0.000000   

        798       799  
0  0.111111  0.000000  
1  0.000000  0.000000  
2  0.111111  0.571429  
3  0.000000  0.142857  
4  0.222222  0.000000  

[5 rows x 800 columns]

0    1
1    1
2    1
3    1
4    1
Name: y, dtype: int64

In [56]:
valid_df = valid_df.sample(frac = 1)

In [58]:
X_valid = valid_df.drop(columns='y')
y_valid = valid_df['y']

In [60]:
Y_valid_predict = model_SVC.predict(X_valid)

In [63]:
f1_score(y_true = y_valid.values, y_pred = Y_valid_predict)
#Previous Iteration: 2.1% - 20%
#After all corrections! v1: 0.994 -> v3: 0.997

0.6609336609336609

In [62]:
accuracy_score(y_true = y_valid.values, y_pred = Y_valid_predict)
#Previous Iteration: 2.1% - 20%
#After all corrections! v1: 0.994 -> v3: 0.997

0.5027027027027027